# California 2045
---
Plots in the slide deck presented to Nathan on 11/27/2018 have been generated through this notebook.

[Base Case](#base)
1. [Chart](#base_chart)
2. [Stacked plots](#base_stack)
3. [Curtailment plots](#base_curtailment)


[California 2020](#ca2020)
1. [Chart](#ca2020_chart)
2. [Stacked plots](#ca2020_stack)
3. [Curtailment plots](#ca2020_curtailment)


[Congestion 2020](#congestion2020)
1. [Chart](#congestion2020_chart)
2. [Stacked plots](#congestion2020_stack)
3. [Curtailment plots](#comgestion2020_curtailment)


[California 2030](#ca2030)
1. [Chart](#ca2030_chart)
2. [Stacked plots](#ca2030_stack)
3. [Curtailment plots](#ca2030_curtailment)


[Congestion 2030](#congestion2030)
1. [Chart](#congestion2030_chart)
2. [Stacked plots](#congestion2030_stack)
3. [Curtailment plots](#comgestion2030_curtailment)


[West 2020](#west2020)
1. [Chart](#west2020_chart)
2. [Stacked plots](#west2020_stack)
3. [Curtailment plots](#west2020_curtailment)


[West 2030](#west2030)
1. [Chart](#west2030_chart)
2. [Stacked plots](#west2030_stack)
3. [Curtailment plots](#west2030_curtailment)

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import analyze
from plot.analyze_pg import AnalyzePG
from westernintnet.westernintnet import win_data
import pandas as pd
import matplotlib.pyplot as plt
plt.ioff()

Download data from server. This needs to be done once per scenario. Since this procedure is lengthy, the data are saved locally as a pickle file.

sftp = analyze.setup_server_connection("U:\\.ssh\\id_rsa")  
scenario_list = analyze.get_scenario_file_from_server(sftp)
scenario = 'california2030_westTarget'
"""  
scenario = 'california2020_westTarget'  
scenario = 'california2030_fixCalCong'  
scenario = 'california2020_fixCalCong'  
scenario = 'california2030Test01'  
scenario = 'california2020Test01'  
scenario = 'western_scenario_Update01'  
"""  
print('scenario: %s' % scenario)  
PG = analyze.get_PG(sftp, scenario_list, scenario)  
PG.to_pickle('U:\\PGData\\California_Nov2018\\PG_CA2030_WestTarget.pkl')

In [ ]:
def get_plant_id(zone, resource):
    plant_id = []
    if zone == 'Western':
        try:
            plant_id = win_data.genbus.groupby('type').get_group(resource).index.values.tolist()
        except KeyError:
            pass
    elif zone == 'California':
        CA = ['Bay Area', 'Central California', 'Northern California',
                'Southeast California', 'Southwest California']
        for load_zone in CA:
            try:
                plant_id += win_data.genbus.groupby(['ZoneName', 'type']).get_group(
                    (load_zone, resource)).index.values.tolist()
            except KeyError:
                pass
    else:
        try:
            plant_id = win_data.genbus.groupby(['ZoneName', 'type']).get_group(
                (zone, resource)).index.values.tolist()
        except KeyError:
            pass
    
    return plant_id

## Base Case
<a id="base"></a>
Multiperiod  
Solar power: SAM  
demand v3  
No multiplier for any of the generator in the grid

In [ ]:
win_data.read_demand_data('v3')
win_data.read_solar_data('sam')
win_data.read_wind_data()

In [ ]:
multiplier = pd.DataFrame({'multiplier': [1]*len(win_data.genbus)}, index=win_data.genbus.index.values)

In [ ]:
PG = pd.read_pickle('U:\\PGData\\California_Nov2018\\PG.pkl')

### 1. Charts
<a id="base_chart"></a>

In [ ]:
chart = AnalyzePG((PG, win_data, multiplier),
    ('2016-01-01-00', '2016-12-31-23', 'utc', 'H'),
    ['California', 'Western'],
    ['nuclear', 'hydro', 'coal', 'ng', 'solar', 'wind'],
    'chart', normalize=False)
chart.get_plot(save=True)
data_chart = chart.get_data()
print("Total Generation in Western Interconnection: %d MWh" % round(data_chart['Western']['Generation'].sum().sum()))
print("Total Capacity in Western Interconnection: %d MW" % round(data_chart['Western']['Capacity'].sum()))

### 2. Stacked plots
<a id="base_stack"></a>

In [ ]:
stack = AnalyzePG((PG, win_data, multiplier),
    ('2016-06-01-00', '2016-06-20-23', 'local', 'H'),
    ['California', 'Western'],
    ['nuclear', 'coal', 'ng', 'hydro', 'wind', 'solar'],
    'stacked', normalize=False)
stack.get_plot(save=False)
data_stack = stack.get_data()
print("Generation in the Western interconnection in MWh:")
print(data_stack['California'].sum().sum())

### 3. Curtailment plots
<a id="base_curtailment"></a>

In [ ]:
curtailment = AnalyzePG((PG, win_data, multiplier),
    ('2016-01-01-00', '2016-12-31-23', 'local', 'D'),
    ['California', 'Western'],
    ['solar', 'wind'],
    'curtailment', normalize=False)
curtailment.get_plot(save=False)
data_curtailment = curtailment.get_data()
print("Maximum curtailment for solar generators in the Western Interconnection: %d %%" % 
      round(data_curtailment['Western']['solar'].curtailment.max()))

## California 2020
<a id="ca2020"></a>
Multiperiod  
Solar power: SAM  
demand v3  
multiplier: 1.52 for solar power output in CA and 1.48 for wind power output in CA.

In [ ]:
win_data.read_demand_data('cal2020')
win_data.read_solar_data('sam')
win_data.read_wind_data()

In [ ]:
factor = [1.52, 1.48]
multiplier = pd.DataFrame({'multiplier': [1]*len(win_data.genbus)}, index=win_data.genbus.index.values)
for i, resource in enumerate(['solar', 'wind']):
    plant_id = get_plant_id('California', resource)
    multiplier.loc[plant_id] = factor[i]

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(12,4))

ax = fig.gca()
multiplier.plot(ax=ax, legend=False)
ax.set_xlabel('Plant id', fontsize=18)
ax.set_ylabel('Multiplier', fontsize=18)
ax.tick_params(labelsize=16)
plt.show()

In [ ]:
PG = pd.read_pickle('U:\\PGData\\California_Nov2018\\PG_CA2020.pkl')

### 1. Charts
<a id="ca2020_chart"></a>

In [ ]:
chart = AnalyzePG((PG, win_data, multiplier),
                  ('2016-01-01-00', '2016-12-31-23', 'utc', 'H'),
                  ['California', 'Western'],
                  ['nuclear', 'hydro', 'coal', 'ng', 'solar', 'wind'],
                  'chart', normalize=False)
chart.get_plot(save=True)
data_chart = chart.get_data()
print("Total Generation in Western Interconnection: %d MWh" % round(data_chart['Western']['Generation'].sum().sum()))
print("Total Capacity in Western Interconnection: %d MW" % round(data_chart['Western']['Capacity'].sum()))

### 2. Stacked Plots
<a id="ca2020_stack"></a>

In [ ]:
stack = AnalyzePG((PG, win_data, multiplier),
                  ('2016-06-01-00', '2016-06-20-23', 'local', 'H'),
                  ['California', 'Western'],
                  ['nuclear', 'coal', 'ng', 'hydro', 'wind', 'solar'],
                  'stacked', normalize=False)
stack.get_plot(save=False)
data_stack = stack.get_data()
print("Generation in the Western interconnection in MWh:")
print(data_stack['California'].sum().sum())

### 3. Curtailment Plots
<a id="ca2020_curtailment"></a>

In [ ]:
curtailment = AnalyzePG((PG, win_data, multiplier),
                        ('2016-01-01-00', '2016-12-31-23', 'local', 'D'),
                        ['California', 'Western'],
                        ['solar', 'wind'],
                        'curtailment', normalize=False)
curtailment.get_plot(save=False)
data_curtailment = curtailment.get_data()

In [ ]:
for zone in ['California', 'Western']:
    for resource in ['solar', 'wind']:
        available = data_curtailment[zone][resource].available.sum() 
        generated = data_curtailment[zone][resource].generated.sum()
        curtailment = 100 * (available - generated) / available
        print("Curtailment %s/%s: %.1f %%" % (zone, resource, curtailment))

## Congestion 2020
<a id="congestion2020"></a>
Multiperiod  
Solar power: SAM  
demand v3  
multiplier: 1.52 for solar power output in CA and 1.48 for wind power output in CA  
Capacity of congested transmission lines in California: x2

In [ ]:
win_data.read_demand_data('cal2020')
win_data.read_solar_data('sam')
win_data.read_wind_data()

In [ ]:
factor = [1.52, 1.48]
multiplier = pd.DataFrame({'multiplier': [1]*len(win_data.genbus)}, index=win_data.genbus.index.values)
for i, resource in enumerate(['solar', 'wind']):
    plant_id = get_plant_id('California', resource)
    multiplier.loc[plant_id] = factor[i]

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(12,4))

ax = fig.gca()
multiplier.plot(ax=ax, legend=False)
ax.set_xlabel('Plant id', fontsize=18)
ax.set_ylabel('Multiplier', fontsize=18)
ax.tick_params(labelsize=16)
plt.show()

In [ ]:
PG = pd.read_pickle('U:\\PGData\\California_Nov2018\\PG_CA2020_CALineCapacityX2.pkl')

### 1. Charts
<a id="congestion2020_chart"></a>

In [ ]:
chart = AnalyzePG((PG, win_data, multiplier),
                  ('2016-01-01-00', '2016-12-31-23', 'utc', 'H'),
                  ['California', 'Western'],
                  ['nuclear', 'hydro', 'coal', 'ng', 'solar', 'wind'],
                  'chart', normalize=False)
chart.get_plot(save=True)
data_chart = chart.get_data()
print("Total Generation in Western Interconnection: %d MWh" % round(data_chart['Western']['Generation'].sum().sum()))
print("Total Capacity in Western Interconnection: %d MW" % round(data_chart['Western']['Capacity'].sum()))

### 2. Stacked Plots
<a id="congestion2020_stack"></a>

In [ ]:
stack = AnalyzePG((PG, win_data, multiplier),
                  ('2016-06-01-00', '2016-06-20-23', 'local', 'H'),
                  ['California', 'Western'],
                  ['nuclear', 'coal', 'ng', 'hydro', 'wind', 'solar'],
                  'stacked', normalize=False)
stack.get_plot(save=False)
data_stack = stack.get_data()
print("Generation in the Western interconnection in MWh:")
print(data_stack['California'].sum().sum())

### 3. Curtailment Plots
<a id="congestion2020_curtailment"></a>

In [ ]:
curtailment = AnalyzePG((PG, win_data, multiplier),
                        ('2016-01-01-00', '2016-12-31-23', 'local', 'D'),
                        ['California', 'Western'],
                        ['solar', 'wind'],
                        'curtailment', normalize=False)
curtailment.get_plot(save=False)
data_curtailment = curtailment.get_data()

In [ ]:
for zone in ['California', 'Western']:
    for resource in ['solar', 'wind']:
        available = data_curtailment[zone][resource].available.sum() 
        generated = data_curtailment[zone][resource].generated.sum()
        curtailment = 100 * (available - generated) / available
        print("Curtailment %s/%s: %.1f %%" % (zone, resource, curtailment))

## California 2030
<a id="ca2030"></a>
Multiperiod  
Solar power: SAM  
demand v3  
multiplier: 2.3 for solar and wind power output for whole western interconnect.

In [ ]:
win_data.read_demand_data('cal2030')
win_data.read_solar_data('sam')
win_data.read_wind_data()

In [ ]:
factor = [2.3, 2.3]
multiplier = pd.DataFrame({'multiplier': [1]*len(win_data.genbus)}, index=win_data.genbus.index.values)
for i, resource in enumerate(['solar', 'wind']):
    plant_id = get_plant_id('Western', resource)
    multiplier.loc[plant_id] = factor[i]

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(12,4))

ax = fig.gca()
multiplier.plot(ax=ax, legend=False)
ax.set_xlabel('Plant id', fontsize=18)
ax.set_ylabel('Multiplier', fontsize=18)
ax.tick_params(labelsize=16)
plt.show()

In [ ]:
PG = pd.read_pickle('U:\\PGData\\California_Nov2018\\PG_CA2030.pkl')

### 1. Charts
<a id="ca2030_chart"></a>

In [ ]:
chart = AnalyzePG((PG, win_data, multiplier),
                  ('2016-01-01-00', '2016-12-31-23', 'utc', 'H'),
                  ['California', 'Western'],
                  ['nuclear', 'hydro', 'coal', 'ng', 'solar', 'wind'],
                  'chart', normalize=False)
chart.get_plot(save=True)
data_chart = chart.get_data()
print("Total Generation in Western Interconnection: %d MWh" % round(data_chart['Western']['Generation'].sum().sum()))
print("Total Capacity in Western Interconnection: %d MW" % round(data_chart['Western']['Capacity'].sum()))

### 2. Stacked Plots
<a id="ca2030_stack"></a>

In [ ]:
stack = AnalyzePG((PG, win_data, multiplier),
                  ('2016-06-01-00', '2016-06-20-23', 'local', 'H'),
                  ['California', 'Western'],
                  ['nuclear', 'coal', 'ng', 'hydro', 'wind', 'solar'],
                  'stacked', normalize=False)
stack.get_plot(save=False)
data_stack = stack.get_data()
print("Generation in the Western interconnection in MWh:")
print(data_stack['California'].sum().sum())

### 3. Curtailment Plots
<a id="ca2030_curtailment"></a>

In [ ]:
curtailment = AnalyzePG((PG, win_data, multiplier),
                        ('2016-01-01-00', '2016-12-31-23', 'local', 'D'),
                        ['California', 'Western'],
                        ['solar', 'wind'],
                        'curtailment', normalize=False)
curtailment.get_plot(save=False)
data_curtailment = curtailment.get_data()

In [ ]:
for zone in ['California', 'Western']:
    for resource in ['solar', 'wind']:
        available = data_curtailment[zone][resource].available.sum() 
        generated = data_curtailment[zone][resource].generated.sum()
        curtailment = 100 * (available - generated) / available
        print("Curtailment %s/%s: %.1f %%" % (zone, resource, curtailment))

## Congestion 2030
<a id="congestion2030"></a>
Multiperiod  
Solar power: SAM  
demand v3  
multiplier: 2.3 for solar and wind power output for whole western interconnect.  
Capacity of congested transmission lines in California: x2

In [ ]:
win_data.read_demand_data('cal2030')
win_data.read_solar_data('sam')
win_data.read_wind_data()

In [ ]:
factor = [2.3, 2.3]
multiplier = pd.DataFrame({'multiplier': [1]*len(win_data.genbus)}, index=win_data.genbus.index.values)
for i, resource in enumerate(['solar', 'wind']):
    plant_id = get_plant_id('California', resource)
    multiplier.loc[plant_id] = factor[i]

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(12,4))

ax = fig.gca()
multiplier.plot(ax=ax, legend=False)
ax.set_xlabel('Plant id', fontsize=18)
ax.set_ylabel('Multiplier', fontsize=18)
ax.tick_params(labelsize=16)
plt.show()

In [ ]:
PG = pd.read_pickle('U:\\PGData\\California_Nov2018\\PG_CA2030_CALineCapacityX2.pkl')

### 1. Charts
<a id="congestion2030_chart"></a>

In [ ]:
chart = AnalyzePG((PG, win_data, multiplier),
                  ('2016-01-01-00', '2016-12-31-23', 'utc', 'H'),
                  ['California', 'Western'],
                  ['nuclear', 'hydro', 'coal', 'ng', 'solar', 'wind'],
                  'chart', normalize=False)
chart.get_plot(save=True)
data_chart = chart.get_data()
print("Total Generation in Western Interconnection: %d MWh" % round(data_chart['Western']['Generation'].sum().sum()))
print("Total Capacity in Western Interconnection: %d MW" % round(data_chart['Western']['Capacity'].sum()))

### 2. Stacked Plots
<a id="congestion2030_stack"></a>

In [ ]:
stack = AnalyzePG((PG, win_data, multiplier),
                  ('2016-06-01-00', '2016-06-20-23', 'local', 'H'),
                  ['California', 'Western'],
                  ['nuclear', 'coal', 'ng', 'hydro', 'wind', 'solar'],
                  'stacked', normalize=False)
stack.get_plot(save=False)
data_stack = stack.get_data()
print("Generation in the Western interconnection in MWh:")
print(data_stack['California'].sum().sum())

### 3. Curtailment Plots
<a id="congestion2030_curtailment"></a>

In [ ]:
curtailment = AnalyzePG((PG, win_data, multiplier),
                        ('2016-01-01-00', '2016-12-31-23', 'local', 'D'),
                        ['California', 'Western'],
                        ['solar', 'wind'],
                        'curtailment', normalize=False)
curtailment.get_plot(save=False)
data_curtailment = curtailment.get_data()

In [ ]:
for zone in ['California', 'Western']:
    for resource in ['solar', 'wind']:
        available = data_curtailment[zone][resource].available.sum() 
        generated = data_curtailment[zone][resource].generated.sum()
        curtailment = 100 * (available - generated) / available
        print("Curtailment %s/%s: %.1f %%" % (zone, resource, curtailment))

## West 2020
<a id="west2020"></a>
Multiperiod  
Solar power: SAM  
demand v3  
multiplier: 2.25 for solar and wind power output for whole western interconnect.

In [ ]:
win_data.read_demand_data('cal2020')
win_data.read_solar_data('sam')
win_data.read_wind_data()

In [ ]:
factor = [2.25, 2.25]
multiplier = pd.DataFrame({'multiplier': [1]*len(win_data.genbus)}, index=win_data.genbus.index.values)
for i, resource in enumerate(['solar', 'wind']):
    plant_id = get_plant_id('Western', resource)
    multiplier.loc[plant_id] = factor[i]

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(12,4))

ax = fig.gca()
multiplier.plot(ax=ax, legend=False)
ax.set_xlabel('Plant id', fontsize=18)
ax.set_ylabel('Multiplier', fontsize=18)
ax.tick_params(labelsize=16)
plt.show()

In [ ]:
PG = pd.read_pickle('U:\\PGData\\California_Nov2018\\PG_CA2020_WestTarget.pkl')

### 1. Charts
<a id="west2020_chart"></a>

In [ ]:
chart = AnalyzePG((PG, win_data, multiplier),
                  ('2016-01-01-00', '2016-12-31-23', 'utc', 'H'),
                  ['California', 'Western'],
                  ['nuclear', 'hydro', 'coal', 'ng', 'solar', 'wind'],
                  'chart', normalize=False)
chart.get_plot(save=True)
data_chart = chart.get_data()
print("Total Generation in Western Interconnection: %d MWh" % round(data_chart['Western']['Generation'].sum().sum()))
print("Total Capacity in Western Interconnection: %d MW" % round(data_chart['Western']['Capacity'].sum()))

### 2. Stacked Plots
<a id="west2020_stack"></a>

In [ ]:
stack = AnalyzePG((PG, win_data, multiplier),
                  ('2016-06-01-00', '2016-06-20-23', 'local', 'H'),
                  ['California', 'Western'],
                  ['nuclear', 'coal', 'ng', 'hydro', 'wind', 'solar'],
                  'stacked', normalize=False)
stack.get_plot(save=False)
data_stack = stack.get_data()
print("Generation in the Western interconnection in MWh:")
print(data_stack['California'].sum().sum())

### 3. Curtailment Plots
<a id="west2020_curtailment"></a>

In [ ]:
curtailment = AnalyzePG((PG, win_data, multiplier),
                        ('2016-01-01-00', '2016-12-31-23', 'local', 'D'),
                        ['California', 'Western'],
                        ['solar', 'wind'],
                        'curtailment', normalize=False)
curtailment.get_plot(save=False)
data_curtailment = curtailment.get_data()

In [ ]:
for zone in ['California', 'Western']:
    for resource in ['solar', 'wind']:
        available = data_curtailment[zone][resource].available.sum() 
        generated = data_curtailment[zone][resource].generated.sum()
        curtailment = 100 * (available - generated) / available
        print("Curtailment %s/%s: %.1f %%" % (zone, resource, curtailment))

## West 2030
<a id="west2030"></a>
Multiperiod  
Solar power: SAM  
demand v3  
multiplier: 5 for solar and wind power output for whole western interconnect.

In [ ]:
win_data.read_demand_data('cal2030')
win_data.read_solar_data('sam')
win_data.read_wind_data()

In [ ]:
factor = [5, 5]
multiplier = pd.DataFrame({'multiplier': [1]*len(win_data.genbus)}, index=win_data.genbus.index.values)
for i, resource in enumerate(['solar', 'wind']):
    plant_id = get_plant_id('Western', resource)
    multiplier.loc[plant_id] = factor[i]

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(12,4))

ax = fig.gca()
multiplier.plot(ax=ax, legend=False)
ax.set_xlabel('Plant id', fontsize=18)
ax.set_ylabel('Multiplier', fontsize=18)
ax.tick_params(labelsize=16)
plt.show()

In [ ]:
PG = pd.read_pickle('U:\\PGData\\California_Nov2018\\PG_CA2030_WestTarget.pkl')

### 1. Charts
<a id="west2030_chart"></a>

In [ ]:
chart = AnalyzePG((PG, win_data, multiplier),
                  ('2016-01-01-00', '2016-12-31-23', 'utc', 'H'),
                  ['California', 'Western'],
                  ['nuclear', 'hydro', 'coal', 'ng', 'solar', 'wind'],
                  'chart', normalize=False)
chart.get_plot(save=True)
data_chart = chart.get_data()
print("Total Generation in Western Interconnection: %d MWh" % round(data_chart['Western']['Generation'].sum().sum()))
print("Total Capacity in Western Interconnection: %d MW" % round(data_chart['Western']['Capacity'].sum()))

### 2. Stacked Plots
<a id="west2030_stack"></a>

In [ ]:
stack = AnalyzePG((PG, win_data, multiplier),
                  ('2016-06-01-00', '2016-06-20-23', 'local', 'H'),
                  ['California', 'Western'],
                  ['nuclear', 'coal', 'ng', 'hydro', 'wind', 'solar'],
                  'stacked', normalize=False)
stack.get_plot(save=False)
data_stack = stack.get_data()
print("Generation in the Western interconnection in MWh:")
print(data_stack['California'].sum().sum())

### 3. Curtailment Plots
<a id="west2030_curtailment"></a>

In [ ]:
curtailment = AnalyzePG((PG, win_data, multiplier),
                        ('2016-01-01-00', '2016-12-31-23', 'local', 'D'),
                        ['California', 'Western'],
                        ['solar', 'wind'],
                        'curtailment', normalize=False)
curtailment.get_plot(save=False)
data_curtailment = curtailment.get_data()

In [ ]:
for zone in ['California', 'Western']:
    for resource in ['solar', 'wind']:
        available = data_curtailment[zone][resource].available.sum() 
        generated = data_curtailment[zone][resource].generated.sum()
        curtailment = 100 * (available - generated) / available
        print("Curtailment %s/%s: %.1f %%" % (zone, resource, curtailment))